# Advanced RAG Pipeline with AWS Bedrock

**Learning Objectives:**
- Build a complete RAG pipeline using LangChain and AWS Bedrock
- Understand document chunking strategies and their impact
- Create and query a vector database with FAISS
- Visualize embeddings in 2D space
- Implement the retrieval-generation workflow

**Inspired by:** [Hugging Face RAG Documentation](https://huggingface.co/learn/cookbook/advanced_rag)

## Setup

In [82]:
!pip install -q langchain langchain-aws faiss-cpu datasets langchain-community boto3 transformers umap-learn plotly umap

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [ ]:
import os
import boto3
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from typing import List, Optional
from langchain_aws import BedrockEmbeddings, ChatBedrock

pd.set_option("display.max_colwidth", None)

## 1. Configure AWS Bedrock

**Models used:**
- **Embeddings**: Titan Text Embeddings v2 (1024 dimensions)
- **LLM**: Nova Lite (fast, cost-effective)

In [ ]:
os.environ["AWS_BEARER_TOKEN_BEDROCK"] = (
    ""
)

In [ ]:
client = boto3.client(service_name="bedrock-runtime", region_name="us-east-1")

EMBEDDING_MODEL_ID = "amazon.titan-embed-text-v2:0"
LLM_MODEL_ID = "amazon.nova-lite-v1:0"

embedding_model = BedrockEmbeddings(
    client=client,
    model_id=EMBEDDING_MODEL_ID,
)

llm = ChatBedrock(
    client=client,
    model_id=LLM_MODEL_ID,
    model_kwargs={
        "max_tokens": 2000,
        "temperature": 0.2,
        "top_p": 0.9,
    },
)

print("✓ AWS Bedrock configured")
print(f"  Embedding model: {EMBEDDING_MODEL_ID}")
print(f"  LLM: {LLM_MODEL_ID}")

## 2. Load Knowledge Base

Using a subset of Hugging Face documentation as our knowledge base.

In [ ]:
import datasets
from langchain.docstore.document import Document

ds = datasets.load_dataset("m-ric/huggingface_doc", split="train[:100]")

knowledge_base = [
    Document(page_content=doc["text"], metadata={"source": doc["source"]})
    for doc in tqdm(ds, desc="Loading documents")
]

print(f"\n✓ Loaded {len(knowledge_base)} documents")
print(f"\nExample document:")
print(f"  Source: {knowledge_base[0].metadata['source']}")
print(f"  Length: {len(knowledge_base[0].page_content)} characters")
print(f"  Preview: {knowledge_base[0].page_content[:200]}...")

## 3. Document Chunking Strategy

### Why Chunk?
- Embedding models have maximum sequence length limits
- Smaller chunks = more focused retrieval
- Too small = lose context; Too large = less precise

### Strategy
We'll use **RecursiveCharacterTextSplitter** with Markdown-aware separators.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    add_start_index=True,
    strip_whitespace=True,
    separators=MARKDOWN_SEPARATORS,
)

docs_processed = []
for doc in tqdm(knowledge_base, desc="Chunking documents"):
    docs_processed.extend(text_splitter.split_documents([doc]))

print(f"\n✓ Created {len(docs_processed)} chunks from {len(knowledge_base)} documents")

### Analyze Chunk Sizes

Let's check if our chunks fit within the model's token limits.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
chunk_lengths = [
    len(tokenizer.encode(doc.page_content))
    for doc in tqdm(docs_processed, desc="Tokenizing")
]

plt.figure(figsize=(10, 5))
pd.Series(chunk_lengths).hist(bins=30, edgecolor="black")
plt.xlabel("Chunk Length (tokens)")
plt.ylabel("Frequency")
plt.title("Distribution of Chunk Lengths")
plt.axvline(512, color="red", linestyle="--", label="Typical model limit (512)")
plt.legend()
plt.show()

print(f"\nChunk length statistics:")
print(f"  Mean: {pd.Series(chunk_lengths).mean():.0f} tokens")
print(f"  Max: {pd.Series(chunk_lengths).max()} tokens")
print(f"  Min: {pd.Series(chunk_lengths).min()} tokens")

### Optimize Chunking with Token-Based Splitting

If chunks are too large, we'll use a tokenizer-based splitter.

In [ ]:
def split_documents_optimized(
    knowledge_base: List[Document],
    chunk_size: int = 512,
    tokenizer_name: str = "bert-base-uncased",
) -> List[Document]:
    """
    Split documents using tokenizer-based chunking.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size=chunk_size,
        chunk_overlap=int(chunk_size / 10),
        add_start_index=True,
        strip_whitespace=True,
        separators=MARKDOWN_SEPARATORS,
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed.extend(text_splitter.split_documents([doc]))

    # Remove duplicates
    seen = set()
    unique_docs = []
    for doc in docs_processed:
        if doc.page_content not in seen:
            seen.add(doc.page_content)
            unique_docs.append(doc)

    return unique_docs


docs_processed = split_documents_optimized(knowledge_base, chunk_size=512)

# Verify the new distribution
chunk_lengths = [
    len(tokenizer.encode(doc.page_content))
    for doc in tqdm(docs_processed, desc="Re-tokenizing")
]

plt.figure(figsize=(10, 5))
pd.Series(chunk_lengths).hist(bins=30, edgecolor="black", color="green", alpha=0.7)
plt.xlabel("Chunk Length (tokens)")
plt.ylabel("Frequency")
plt.title("Optimized Chunk Length Distribution")
plt.axvline(512, color="red", linestyle="--", label="Target limit (512)")
plt.legend()
plt.show()

print(f"\n✓ Optimized to {len(docs_processed)} chunks")
print(f"  Max length: {max(chunk_lengths)} tokens (within limit!)")

## 4. Build Vector Database with FAISS

FAISS (Facebook AI Similarity Search) is a fast library for similarity search.

In [ ]:
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

print("Creating FAISS vector database...")
print("(This may take a few minutes to embed all documents)\n")

vector_db = FAISS.from_documents(
    docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
)

print(f"✓ Vector database created")
print(f"  Total vectors: {vector_db.index.ntotal}")
print(f"  Dimension: {vector_db.index.d}")

## 5. Visualize Embeddings in 2D

Let's visualize how documents and queries are positioned in embedding space.

In [80]:
user_query = "How to create a pipeline object?"
query_vector = embedding_model.embed_query(user_query)

print(f"Query: {user_query}")
print(f"Query embedding dimension: {len(query_vector)}")

Query: How to create a pipeline object?
Query embedding dimension: 1024


In [81]:
import numpy as np
from umap import UMAP

# Collect all embeddings (documents + query)
n_docs = len(docs_processed)
all_embeddings = []

for idx in range(n_docs):
    doc_vector = vector_db.index.reconstruct(idx)
    all_embeddings.append(doc_vector)

all_embeddings.append(query_vector)
embeddings_array = np.array(all_embeddings)

# Reduce to 2D using UMAP
print("Reducing dimensions with UMAP...")
n_neighbors = min(15, max(2, n_docs - 1))
reducer = UMAP(n_neighbors=n_neighbors, n_components=2, random_state=42, min_dist=0.1)
embeddings_2d = reducer.fit_transform(embeddings_array)

print(f"✓ Reduced from {embeddings_array.shape[1]}D to 2D")

ModuleNotFoundError: No module named 'umap'

In [ ]:
import plotly.graph_objects as go

# Separate documents and query
doc_x, doc_y = embeddings_2d[:-1, 0], embeddings_2d[:-1, 1]
query_x, query_y = embeddings_2d[-1, 0], embeddings_2d[-1, 1]

fig = go.Figure()

# Documents
fig.add_trace(
    go.Scatter(
        x=doc_x,
        y=doc_y,
        mode="markers",
        name="Documents",
        marker=dict(size=6, color="lightblue", opacity=0.6),
        text=[f"Doc {i}" for i in range(len(doc_x))],
        hovertemplate="<b>%{text}</b><extra></extra>",
    )
)

# Query
fig.add_trace(
    go.Scatter(
        x=[query_x],
        y=[query_y],
        mode="markers",
        name="Query",
        marker=dict(size=15, color="red", symbol="star"),
        text=[user_query],
        hovertemplate="<b>Query:</b> %{text}<extra></extra>",
    )
)

fig.update_layout(
    title="2D Visualization of Document Embeddings",
    xaxis_title="Dimension 1",
    yaxis_title="Dimension 2",
    width=900,
    height=600,
    template="plotly_white",
)

fig.show()

print(f"\n📊 Red star = query | Blue dots = documents")
print(f"Closer documents are semantically more similar to the query")

## 6. Retrieval: Query the Vector Database

In [ ]:
print(f"Query: {user_query}\n")

retrieved_docs = vector_db.similarity_search(query=user_query, k=5)

print("=" * 80)
print("TOP 5 RETRIEVED DOCUMENTS")
print("=" * 80)

for i, doc in enumerate(retrieved_docs, 1):
    print(f"\n[Rank {i}]")
    print(f"Source: {doc.metadata.get('source', 'Unknown')}")
    print(f"Content: {doc.page_content[:300]}...")
    print("-" * 80)

## 7. Generation: LLM Response with Retrieved Context

Now we'll use the retrieved documents as context for the LLM to generate an answer.

In [ ]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a helpful assistant. Use the provided context to answer the user's question.
        
Rules:
- Answer only based on the context provided
- Be concise and relevant
- Cite document numbers when referencing information
- If the answer is not in the context, say so""",
        ),
        (
            "human",
            """Context:
{context}

Question: {question}""",
        ),
    ]
)

print("✓ RAG prompt template created")

In [ ]:
# Prepare context from retrieved documents
context = "\n\n".join(
    [f"Document {i}:\n{doc.page_content}" for i, doc in enumerate(retrieved_docs, 1)]
)

# Format prompt
messages = RAG_PROMPT.format_messages(question=user_query, context=context)

# Generate answer
print("Generating answer...\n")
response = llm.invoke(messages)

print("=" * 80)
print("RAG ANSWER")
print("=" * 80)
print(response.content)
print("=" * 80)

## 8. Complete RAG Pipeline Function

In [ ]:
def rag_query(query: str, k: int = 5) -> dict:
    """
    Complete RAG pipeline: retrieve relevant docs and generate answer.

    Args:
        query: User's question
        k: Number of documents to retrieve

    Returns:
        Dict with answer, retrieved docs, and metadata
    """
    # Retrieve
    retrieved_docs = vector_db.similarity_search(query=query, k=k)

    # Prepare context
    context = "\n\n".join(
        [
            f"Document {i}:\n{doc.page_content}"
            for i, doc in enumerate(retrieved_docs, 1)
        ]
    )

    # Generate
    messages = RAG_PROMPT.format_messages(question=query, context=context)
    response = llm.invoke(messages)

    return {
        "query": query,
        "answer": response.content,
        "retrieved_docs": retrieved_docs,
        "num_docs": len(retrieved_docs),
    }


# Test the pipeline
test_queries = [
    "How to create a pipeline object?",
    "What is the purpose of tokenizers?",
    "How to fine-tune a model?",
]

for query in test_queries:
    print(f"\nQuery: {query}")
    result = rag_query(query, k=3)
    print(f"Answer: {result['answer'][:200]}...")
    print("-" * 80)

## 9. Key Takeaways

### RAG Pipeline Components

1. **Document Processing**
   - Load knowledge base
   - Chunk documents (token-aware)
   - Remove duplicates

2. **Indexing**
   - Embed documents
   - Store in vector database (FAISS)
   - Enable fast similarity search

3. **Retrieval**
   - Embed query
   - Find k most similar documents
   - Return relevant context

4. **Generation**
   - Format retrieved docs as context
   - Create prompt for LLM
   - Generate grounded answer

### Best Practices

- Use tokenizer-based chunking for consistent sizes
- Choose chunk size based on model limits and task
- Optimize k based on context window and relevance
- Monitor chunk distribution to avoid truncation
- Use appropriate distance metrics (cosine for normalized embeddings)